In [47]:
import pickle as pkl
import torch
import os

In [44]:
def get_data(prefix, base_path='./'):
    in_dir = os.listdir(base_path)
    in_dir = [f for f in in_dir if f.startswith(prefix)]
    in_dir = sorted(in_dir)
    data = []
    for f in in_dir:
        with open(base_path + f, 'rb') as file:
            data.append(pkl.load(file))
    return data

In [46]:
tedlium_test = get_data('ted_test', base_path='./online/')
wer_tedlium_test = [d['wer'] for d in tedlium_test]
print(f'{len(wer_tedlium_test)}/3 complete')
sum(wer_tedlium_test)/len(wer_tedlium_test)

3/3 complete


0.061210546005867644

In [25]:
tedlium_test[0].keys()

dict_keys(['wer', 'words', 'ins_rate', 'del_rate', 'sub_rate', 'model_output', 'gold', 'args_dict', 'repeat'])

In [22]:
tedlium_test[0]['ins_rate']

0.008683324472798157

In [20]:
import torch, lcasr

In [27]:
lcasr.models.sconformer_xl.SCConformerXL

lcasr.models.sconformer_xl.SCConformerXL

In [31]:
model_path = "/store/store5/data/acp21rjf_checkpoints/lcasr/lcasr-160-rb_e1/step_105360.pt"
model_state = torch.load(model_path, map_location='cpu')
model = lcasr.models.sconformer_xl.SCConformerXL(vocab_size=4095, use_rotary=True, decoder_norm=True)
model.load_state_dict(model_state['model'])

<All keys matched successfully>

In [42]:
from apex.normalization import FusedLayerNorm
from lcasr.components.batchrenorm import BatchRenorm1d

In [43]:

for param in model.parameters():
    param.requires_grad = False
for module in model.modules():
    if isinstance(module, FusedLayerNorm) or isinstance(module, torch.nn.LayerNorm):
        module.bias.requires_grad = True
    if isinstance(module, torch.nn.Linear) and module.bias is not None:
        module.bias.requires_grad = True
    if isinstance(module, BatchRenorm1d):
        module.bias.requires_grad = True

0 BatchRenorm1d()
0 BatchRenorm1d()
0 BatchRenorm1d()
0 BatchRenorm1d()
0 BatchRenorm1d()
0 BatchRenorm1d()
